In [9]:
import findspark
findspark.init('/usr/local/spark')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [12]:
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

3.1414656


In [19]:
sc = pyspark.SparkContext(master = 'local[*]')
sc.defaultParallelism
sc.stop()

16

In [20]:
list(map(lambda x: x ** 3, [1, 2, 3, 4, 5]))

[1, 8, 27, 64, 125]

In [37]:
sc = pyspark.SparkContext(master = 'local[*]')
x = sc.parallelize([1, 2, 3, 4, 5]).map(lambda x: x ** 3).collect()
sc.stop()
x

[1, 8, 27, 64, 125]

In [15]:
sc = pyspark.SparkContext(master = 'local[*]')
sc.defaultParallelism
sc.stop()

16

# Feature tools

In [66]:
import featuretools as ft
import featuretools.variable_types as vtypes 

feature_defs = ft.load_features('/data/churn/features.txt')
print(f'There are {len(feature_defs)} features.')

There are 230 features.


In [72]:
import pandas as pd
import numpy as np

N_PARTITIONS = 1000

In [143]:
def partition_to_feature_matrix(partition, feature_defs=feature_defs):
    """Take in a partition number and return a feature matrix"""
    directory = '/data/churn/partitions/p' + str(partition)
    
    # Read in the data files
    members = pd.read_csv(f'{directory}/members.csv', 
                      parse_dates=['registration_init_time'], 
                      infer_datetime_format = True, 
                      dtype = {'gender': 'category'})

    trans = pd.read_csv(f'{directory}/transactions.csv',
                       parse_dates=['transaction_date', 'membership_expire_date'], 
                        infer_datetime_format = True)

    logs = pd.read_csv(f'{directory}/logs.csv', parse_dates = ['date'])
    cutoff_times = pd.read_csv(f'{directory}/cutoff_times.csv', parse_dates = ['cutoff'])
    cutoff_times = cutoff_times.drop_duplicates()
    
    # Create empty entityset
    es = ft.EntitySet(id = 'customers')

    # Add the members parent table
    es.entity_from_dataframe(entity_id='members', dataframe=members,
                             index = 'msno', time_index = 'registration_init_time', 
                             variable_types = {'city': vtypes.Categorical, 'bd': vtypes.Categorical,
                                               'registered_via': vtypes.Categorical})
    # Create new features in transactions
    trans['price_difference'] = trans['plan_list_price'] - trans['actual_amount_paid']
    trans['planned_daily_price'] = trans['plan_list_price'] / trans['payment_plan_days']
    trans['daily_price'] = trans['actual_amount_paid'] / trans['payment_plan_days']

    # Add the transactions child table
    es.entity_from_dataframe(entity_id='transactions', dataframe=trans,
                             index = 'transactions_index', make_index = True,
                             time_index = 'transaction_date', 
                             variable_types = {'payment_method_id': vtypes.Categorical, 
                                               'is_auto_renew': vtypes.Boolean, 'is_cancel': vtypes.Boolean})

    # Add transactions interesting values
    es['transactions']['is_cancel'].interesting_values = [0, 1]
    es['transactions']['is_auto_renew'].interesting_values = [0, 1]
    
    # Create new features in logs
    logs['total'] = logs[['num_25', 'num_50', 'num_75', 'num_985', 'num_100']].sum(axis = 1)
    logs['percent_100'] = logs['num_100'] / logs['total']
    logs['percent_unique'] = logs['num_unq'] / logs['total']
    
    # Add the logs child table
    es.entity_from_dataframe(entity_id='logs', dataframe=logs,
                         index = 'logs_index', make_index = True,
                         time_index = 'date')

    # Add the relationships
    r_member_transactions = ft.Relationship(es['members']['msno'], es['transactions']['msno'])
    r_member_logs = ft.Relationship(es['members']['msno'], es['logs']['msno'])
    es.add_relationships([r_member_transactions, r_member_logs])

    # Calculate and save the feature matrix
    feature_matrix = ft.calculate_feature_matrix(entityset=es, features=feature_defs, cutoff_time=cutoff_times)
    
    feature_matrix.to_csv(f'{directory}/feature_matrix.csv')
    
    # Report progress every 10th of number of partitions
    if (partition % (N_PARTITIONS / 10) == 0):
        print(f'{100 * round(partition / N_PARTITIONS)}% complete.', end = '\r')

In [144]:
import os
base_dir = '/data/churn/partitions/'
partitions = list(range(len(os.listdir(base_dir))))
partitions[-1]

999

In [145]:
sc.stop()

In [114]:
conf = pyspark.SparkConf()
conf.set('spark.eventLog.enabled', True);
conf.set('spark.eventLog.dir', '/usr/local/spark/tmp');
conf.getAll()

[('spark.eventLog.dir', '/usr/local/spark/tmp'),
 ('spark.eventLog.enabled', 'True'),
 ('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell')]

In [119]:
sc = pyspark.SparkContext(master = 'spark://ip-172-31-23-133.ec2.internal:7077', 
                          conf = conf,
                          appName='Cubed')
sc.parallelize([1, 2, 3, 4, 5], numSlices=1).map(lambda x: x ** 3).collect()
sc.stop()

[1, 8, 27, 64, 125]

In [142]:
df = pd.read_json('/usr/local/spark/tmp/app-20180917145714-0007', lines = True)
df.head()

,App ID,App Name,Block Manager ID,Classpath Entries,Completion Time,Event,Executor ID,Executor Info,JVM Information,Job ID,Job Result,Maximum Memory,Maximum Offheap Memory,Maximum Onheap Memory,Properties,Spark Properties,Spark Version,Stage Attempt ID,Stage ID,Stage IDs,Stage Info,Stage Infos,Submission Time,System Properties,Task End Reason,Task Info,Task Metrics,Task Type,Timestamp,User
0,NaN,NaN,NaN,NaN,NaN,SparkListenerLogStart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.3.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,NaN,NaN,"{'Executor ID': 'driver', 'Host': 'ip-172-31-2...",NaN,NaN,SparkListenerBlockManagerAdded,NaN,NaN,NaN,NaN,NaN,411775795.0,0.0,411775795.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-17 14:57:14.450,NaN
2,NaN,NaN,NaN,{'/usr/local/spark/jars/commons-lang-2.6.jar':...,NaN,SparkListenerEnvironmentUpdate,NaN,NaN,{'Java Home': '/home/ubuntu/anaconda3/envs/pyt...,NaN,NaN,NaN,NaN,NaN,NaN,{'spark.driver.host': 'ip-172-31-23-133.ec2.in...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'java.io.tmpdir': '/tmp', 'line.separator': '...",NaN,NaN,NaN,NaN,NaT,NaN
3,app-20180917145714-0007,Cubed,NaN,NaN,NaN,SparkListenerApplicationStart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-17 14:57:14.417,ubuntu
4,NaN,NaN,NaN,NaN,NaN,SparkListenerJobStart,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"{'spark.rdd.scope.noOverride': 'true', 'callSi...",NaN,NaN,NaN,NaN,[0],NaN,"[{'Stage ID': 0, 'Stage Attempt ID': 0, 'Stage...",1.537196e+12,NaN,NaN,NaN,NaN,NaN,NaT,NaN


## Testing 

In [146]:
from timeit import default_timer as timer

In [150]:
sc.stop()

In [151]:
start = timer()
sc = pyspark.SparkContext(master = 'local[*]', 
                          appName = 'featuretools', conf = conf)
r = sc.parallelize(partitions, numSlices=1000).map(partition_to_feature_matrix).collect()
sc.stop()
end = timer()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 0.0 failed 1 times, most recent failure: Lost task 9.0 in stage 0.0 (TID 9, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 230, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 225, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 372, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 55, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-143-4bd6d5ccd8af>", line 59, in partition_to_feature_matrix
  File "/home/ubuntu/anaco

# Read in Results

In [124]:
df = pd.read_json('/usr/local/spark/tmp/app-20180917145854-0009', lines = True)
df.head()

,App ID,App Name,Block Manager ID,Classpath Entries,Completion Time,Event,Executor ID,Executor Info,JVM Information,Job ID,...,Stage Info,Stage Infos,Submission Time,System Properties,Task End Reason,Task Info,Task Metrics,Task Type,Timestamp,User
0,NaN,NaN,NaN,NaN,NaN,SparkListenerLogStart,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,NaN,NaN,"{'Executor ID': 'driver', 'Host': 'ip-172-31-2...",NaN,NaN,SparkListenerBlockManagerAdded,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-17 14:58:54.536,NaN
2,NaN,NaN,NaN,{'/usr/local/spark/jars/commons-lang-2.6.jar':...,NaN,SparkListenerEnvironmentUpdate,NaN,NaN,{'Java Home': '/home/ubuntu/anaconda3/envs/pyt...,NaN,...,NaN,NaN,NaN,"{'java.io.tmpdir': '/tmp', 'line.separator': '...",NaN,NaN,NaN,NaN,NaT,NaN
3,app-20180917145854-0009,testing,NaN,NaN,NaN,SparkListenerApplicationStart,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-09-17 14:58:54.487,ubuntu
4,NaN,NaN,NaN,NaN,NaN,SparkListenerJobStart,NaN,NaN,NaN,0.0,...,NaN,"[{'Stage ID': 0, 'Stage Attempt ID': 0, 'Stage...",1.537196e+12,NaN,NaN,NaN,NaN,NaN,NaT,NaN


The next code block finds the time in seconds to complete each task.

In [130]:
def filter_task_info(task_dict):
    try:
        return task_dict.get('Task ID')
    except:
        return np.nan
    
df['Task ID'] = df['Task Info'].apply(filter_task_id)

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
5         NaN
6         NaN
7         0.0
8         1.0
9         2.0
10        3.0
11        4.0
12        5.0
13        6.0
14        7.0
15        8.0
16        9.0
17       10.0
18       11.0
19       12.0
20       13.0
21       14.0
22       15.0
23        NaN
24       16.0
25       12.0
26       17.0
27        4.0
28       18.0
29       10.0
        ...  
1981    978.0
1982    995.0
1983    981.0
1984    996.0
1985    980.0
1986    997.0
1987    979.0
1988    998.0
1989    983.0
1990    999.0
1991    985.0
1992    984.0
1993    982.0
1994    986.0
1995    987.0
1996    988.0
1997    989.0
1998    992.0
1999    990.0
2000    991.0
2001    993.0
2002    994.0
2003    995.0
2004    999.0
2005    997.0
2006    996.0
2007    998.0
2008      NaN
2009      NaN
2010      NaN
Name: Task ID, Length: 2011, dtype: float64

In [134]:
pd.options.display.max_columns = 40
df[df['Task ID'] == 996]

,App ID,App Name,Block Manager ID,Classpath Entries,Completion Time,Event,Executor ID,Executor Info,JVM Information,Job ID,Job Result,Maximum Memory,Maximum Offheap Memory,Maximum Onheap Memory,Properties,Spark Properties,Spark Version,Stage Attempt ID,Stage ID,Stage IDs,Stage Info,Stage Infos,Submission Time,System Properties,Task End Reason,Task Info,Task Metrics,Task Type,Timestamp,User,Task ID
1984,NaN,NaN,NaN,NaN,NaN,SparkListenerTaskStart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,"{'Task ID': 996, 'Index': 996, 'Attempt': 0, '...",NaN,NaN,NaT,NaN,996.0
2006,NaN,NaN,NaN,NaN,NaN,SparkListenerTaskEnd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,{'Reason': 'Success'},"{'Task ID': 996, 'Index': 996, 'Attempt': 0, '...","{'Executor Deserialize Time': 0, 'Executor Des...",ResultTask,NaT,NaN,996.0


In [135]:
df.loc[1984, 'Task Info']

{'Task ID': 996,
 'Index': 996,
 'Attempt': 0,
 'Launch Time': 1537196517065,
 'Executor ID': '0',
 'Host': '172.31.23.133',
 'Locality': 'PROCESS_LOCAL',
 'Speculative': False,
 'Getting Result Time': 0,
 'Finish Time': 0,
 'Failed': False,
 'Killed': False,
 'Accumulables': []}

In [138]:
df.loc[2006, 'Task Info']['Finish Time'] - df.loc[2006, 'Task Info']['Launch Time']

2809

In [141]:
pd.to_datetime(1537196517065, unit = 'ms')

Timestamp('2018-09-17 15:01:57.065000')